In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from pathlib import Path
from skimage.io import imread
from keras.models import Sequential, Model
from tensorflow.keras import layers
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import to_categorical

In [ ]:
# Define path to the data directory
data_dir = Path('../input/dogs-cats-images/dataset')

# Path to train directory
train_dir = data_dir / 'training_set'

# Path to test directory
test_dir = data_dir / 'test_set'

!wget https://github.com/fchollet/deep-learning-models/releases/download/v0.5/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
cats_dir = train_dir / 'cats'
dogs_dir = train_dir / 'dogs'

# Get the list of all the images
cats = cats_dir.glob('*.jpg')
dogs = dogs_dir.glob('*.jpg')

# An empty list. We will insert the data into this list in (img_path, label) format
train_data = []

# Go through all the cats. The label for these cases will be 0
for img in cats:
    train_data.append((img,0))

# Go through all the dogs. The label for these cases will be 1
for img in dogs:
    train_data.append((img, 1))

# Get a pandas dataframe from the data we have in our list 
train_data = pd.DataFrame(train_data, columns=['image', 'label'],index=None)

# Shuffle the data 
train_data = train_data.sample(frac=1.).reset_index(drop=True)

train_data.head()

In [ ]:
#count for each class
pets_count = train_data['label'].value_counts()
print(pets_count)

#plot the results
plt.figure(figsize=(10,8))
sns.barplot(x=pets_count.index, y = pets_count.values)
plt.title("Number of pets", fontsize=14)
plt.xlabel("Pet", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.xticks(range(len(pets_count.index)), ["Cat(0)", "Dog(1)"])
plt.show()

In [ ]:
cats_samples = (train_data[train_data['label']==0]['image'].iloc[:5]).tolist()
dogs_samples = (train_data[train_data["label"]==1]['image'].iloc[:5]).tolist()
samples = cats_samples + dogs_samples

f, ax = plt.subplots(2,5, figsize=(30,10))
for i in range(10):
    img = imread(samples[i])
    ax[i//5, i%5].imshow(img, cmap='gray')
    if i < 5:
        ax[i//5, i%5].set_title("Cat")
    else:
        ax[i//5, i%5].set_title("Dog")
    ax[i//5, i%5].axis('off')
    ax[i//5, i%5].set_aspect('auto')
plt.show()

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

weights1="./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(weights1)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
pre_trained_model.summary()



In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
x = layers.Flatten()(last_output)

x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(1, activation='sigmoid')(x)           

model = Model(pre_trained_model.input, x) 

opt = Adam(lr=0.001, decay=1e-5)
es = EarlyStopping(patience=5)
checkpoint = ModelCheckpoint(filepath='best_model_todate', save_best_only=True, save_weights_only=True)
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=opt)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)


test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

validation_generator =  test_datagen.flow_from_directory( test_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

In [ ]:
# Fit the model
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 100,
            validation_steps = 50,
            callbacks=[es, checkpoint],
            verbose = 2)

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc, label='Training')
plt.plot  ( epochs, val_acc, label='Validation')
plt.title ('Training and validation accuracy')
plt.legend()
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss, label='Training')
plt.plot  ( epochs, val_loss, label='Validation')
plt.legend()
plt.title ('Training and validation loss')

In [ ]:
#save model
saved_model_path = "./cat_dog_model.h5"
model.save(saved_model_path)


In [ ]:
plt.figure(figsize=(10,15))
for j in range(0,5):
    for i in range(0,10):
        plt.subplot(10,10,j*10+i+1)
        plt.title('%d'%y_test_pred_labels[mn][j*10+i])
        plt.imshow(x_test[j*10+i].reshape(28,28), cmap=cm.binary)

In [ ]:
!wget https://i.ibb.co/sCLbbsF/faisca.jpg
!wget https://i.ibb.co/zRXPNB2/mufasa.png
!wget https://i.ibb.co/4SN6cBp/neo.png

In [ ]:

import matplotlib.pyplot as plt
import cv2


image = './faisca.jpg'
img = cv2.imread(image,0)
img = cv2.resize(img, (150, 150))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img.astype(np.float32)/255.
img_processed = np.expand_dims(img, axis=0)
plt.imshow(img_processed)
#predictions_single = model.predict(img_processed)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(plt.imread(image))
img_processed = ImageDataGenerator(rescale = 1.0/255.)
predictions_single = model.predict(img_processed)

In [ ]:

test_generator =  test_datagen.flow_from_directory( test_dir,
                                                    batch_size  = 20,
                                                    class_mode  = 'binary', 
                                                    target_size = (150, 150))